In [5]:
import pylibmagic
import magic
import biosignalsnotebooks as bsnb
from ast import literal_eval
from h5py import File
from numpy import array, linspace
from numpy import loadtxt, array, mean, logical_and, trapz
from scipy.signal import spectrogram, welch
import pandas as pd
import csv

/Users/hamza/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
with  open('data.txt', "r", encoding="utf8") as file_txt:
    txt_data = file_txt.readlines()
    metadata = txt_data[1]
file_txt.close()
metadata_aux = metadata[2:-1]
header_txt = literal_eval(metadata_aux)
header_txt

data = loadtxt('data.txt')
df = pd.DataFrame(data)
df

ch = 2 # Channel
sr = header_txt.get('00:07:80:8C:06:69').get('sampling rate') # Sampling rate
resolution = header_txt.get('00:07:80:8C:06:69').get('resolution')[2] # Resolution (number of available bits)

signal = df[ch]
vcc = 3e6 # uV
gain = 40000

signal_uv = (((array(signal) / 2**resolution) - 0.5) * vcc) / gain

In [12]:
time = bsnb.generate_time(signal_uv, sr)

In [13]:
#Spectrogram function, using scipy.signal.spectrogram
f, t, Sxx = spectrogram(signal_uv, fs = sr, nfft = 2000)

In [42]:
# Time window of closed eyes
t_closed_start = 500 # lower limit of time window (s)
sample_closed_start = t_closed_start*sr

t_closed_end = 1010 # Upper limit of time window (s)
sample_closed_end = t_closed_end*sr
sample_closed_end

3030000

In [43]:
# Cuttoff frequencies
f1 = 3 # lower cutoff frequency for bandpass filter (Hz)
f2 = 30 # Upper cutoff frequency for bandpass filter (Hz)

# Baseline shift of window
signal_shift_window_eyes_closed = array(signal_uv[sample_closed_start:sample_closed_end]) - mean(array(signal_uv[sample_closed_start:sample_closed_end]))

In [44]:
# Digital Bandpass filtering with cutoff frequencies of f1=3 and f2=30 Hz using bsns.bandpass
filtered_signal_closed_eyes = bsnb.bandpass(signal_shift_window_eyes_closed, f1, f2, order = 2, fs = sr)

In [45]:
# Time window of opened eyes
t_opened_start = 1 # lower limit of time window (s)
sample_opened_start = t_opened_start*sr

t_opened_end = 500 # Upper limit of time window (s)
sample_opened_end = t_opened_end*sr

In [46]:
#Baseline shift of window
signal_shift_window_eyes_opened = array(signal_uv[sample_opened_start:sample_opened_end])-mean(array(signal_uv[sample_opened_start:sample_opened_end]))

#Digital Bandpass filtering with cutoff frequencies of f1=3 and f2=30 Hz using bsns.bandpass
filtered_signal_opened_eyes = bsnb.bandpass(signal_shift_window_eyes_opened, f1, f2, order = 2, fs = sr)

In [47]:
#Time Windows for Welchs method 
win = 4 * sr # 4 seconds time windows.

#FFT with time windows using scipy.signal.welch
freq_axis_eyes_closed, power_spect_eyes_closed = welch(filtered_signal_closed_eyes, sr, nperseg=win)

In [48]:
#FFT with time windows using scipy.signal.welch for open eyes time window
freq_axis_eyes_opened, power_spect_eyes_opened = welch(filtered_signal_opened_eyes, sr, nperseg=win)

In [68]:
#Define Frequency Band limits:
freq_low = 8 #lower limit for alpha band
freq_high = 12 #Upper limit for alpha band 

In [69]:
#Find the intersection Values of the alpha band in the frequency vector [Eyes Closed]
idx_alpha_eyes_closed = logical_and(freq_axis_eyes_closed >= freq_low, freq_axis_eyes_closed <= freq_high)

#Find the intersection Values of the alpha band in the frequency vector [Eyes Opened]
idx_alpha_eyes_opened = logical_and(freq_axis_eyes_opened >= freq_low, freq_axis_eyes_opened <= freq_high)

In [70]:
#Frequency Resolution
freq_res_closed_eyes = freq_axis_eyes_closed[1] - freq_axis_eyes_closed[0] 

#Compute the Absolute Power with numpy.trapz:
alpha_power_closed_eyes = trapz(power_spect_eyes_closed[idx_alpha_eyes_closed],dx=freq_res_closed_eyes)

IndexError: index 1 is out of bounds for axis 0 with size 0

In [37]:
#Frequency Resolution
freq_res_opened_eyes = freq_axis_eyes_opened[1] - freq_axis_eyes_opened[0] 

#Compute the Absolute Power with numpy.trapz:
alpha_power_opened_eyes = trapz(power_spect_eyes_opened[idx_alpha_eyes_opened],dx=freq_res_opened_eyes)

IndexError: index 1 is out of bounds for axis 0 with size 0

In [38]:
alpha_power_opened_eyes

4.396160123537454

In [39]:
alpha_power_closed_eyes

4.396160123537454

In [7]:
def getAlphaPowers(fileName):
    with  open('data.txt', "r", encoding="utf8") as file_txt:
        txt_data = file_txt.readlines()
        metadata = txt_data[1]
        file_txt.close()
        metadata_aux = metadata[2:-1]
        header_txt = literal_eval(metadata_aux)
        header_txt

    data = loadtxt('data.txt')
    df = pd.DataFrame(data)
    df

    ch = 2 # Channel
    sr = header_txt.get('00:07:80:8C:06:69').get('sampling rate') # Sampling rate
    resolution = header_txt.get('00:07:80:8C:06:69').get('resolution')[2] # Resolution (number of available bits)

    signal = df[ch]
    vcc = 3e6 # uV
    gain = 40000

    signal_uv = (((array(signal) / 2**resolution) - 0.5) * vcc) / gain
    
    time = bsnb.generate_time(signal_uv, sr)
    
    f, t, Sxx = spectrogram(signal_uv, fs = sr, nfft = 2000)
    
    # Time window of closed eyes
    t_closed_start = 1 # lower limit of time window (s)
    sample_closed_start = t_closed_start*sr

    t_closed_end = 1010 # Upper limit of time window (s)
    sample_closed_end = t_closed_end*sr
    sample_closed_end
    
    # Cuttoff frequencies
    f1 = 3 # lower cutoff frequency for bandpass filter (Hz)
    f2 = 30 # Upper cutoff frequency for bandpass filter (Hz)

    # Baseline shift of window
    signal_shift_window_eyes_closed = array(signal_uv[sample_closed_start:sample_closed_end]) - mean(array(signal_uv[sample_closed_start:sample_closed_end]))
    
    # Digital Bandpass filtering with cutoff frequencies of f1=3 and f2=30 Hz using bsns.bandpass
    filtered_signal_closed_eyes = bsnb.bandpass(signal_shift_window_eyes_closed, f1, f2, order = 2, fs = sr)
    
    # Time window of opened eyes
    t_opened_start = 1 # lower limit of time window (s)
    sample_opened_start = t_opened_start*sr

    t_opened_end = 1010 # Upper limit of time window (s)
    sample_opened_end = t_opened_end*sr
    
    #Baseline shift of window
    signal_shift_window_eyes_opened = array(signal_uv[sample_opened_start:sample_opened_end])-mean(array(signal_uv[sample_opened_start:sample_opened_end]))

    #Digital Bandpass filtering with cutoff frequencies of f1=3 and f2=30 Hz using bsns.bandpass
    filtered_signal_opened_eyes = bsnb.bandpass(signal_shift_window_eyes_opened, f1, f2, order = 2, fs = sr)
    
    #Time Windows for Welchs method 
    win = 4 * sr # 4 seconds time windows.

    #FFT with time windows using scipy.signal.welch
    freq_axis_eyes_closed, power_spect_eyes_closed = welch(filtered_signal_closed_eyes, sr, nperseg=win)
    
    #FFT with time windows using scipy.signal.welch for open eyes time window
    freq_axis_eyes_opened, power_spect_eyes_opened = welch(filtered_signal_opened_eyes, sr, nperseg=win)
    
    #Define Frequency Band limits:
    freq_low = 8 #lower limit for alpha band
    freq_high = 12 #Upper limit for alpha band 
    
    #Find the intersection Values of the alpha band in the frequency vector [Eyes Closed]
    idx_alpha_eyes_closed = logical_and(freq_axis_eyes_closed >= freq_low, freq_axis_eyes_closed <= freq_high)

    #Find the intersection Values of the alpha band in the frequency vector [Eyes Opened]
    idx_alpha_eyes_opened = logical_and(freq_axis_eyes_opened >= freq_low, freq_axis_eyes_opened <= freq_high)
    
    #Frequency Resolution
    freq_res_closed_eyes = freq_axis_eyes_closed[1] - freq_axis_eyes_closed[0] 

    #Compute the Absolute Power with numpy.trapz:
    alpha_power_closed_eyes = trapz(power_spect_eyes_closed[idx_alpha_eyes_closed],dx=freq_res_closed_eyes) 
    
    #Frequency Resolution
    freq_res_opened_eyes = freq_axis_eyes_opened[1] - freq_axis_eyes_opened[0] 

    #Compute the Absolute Power with numpy.trapz:
    alpha_power_opened_eyes = trapz(power_spect_eyes_opened[idx_alpha_eyes_opened],dx=freq_res_opened_eyes)
    return alpha_power_closed_eyes, alpha_power_opened_eyes
    

In [8]:
alpha_power_closed_eyes, alpha_power_opened_eyes = getAlphaPowers('data.txt')

In [9]:
alpha_power_closed_eyes

4.396160123537454

In [10]:
alpha_power_opened_eyes

4.396160123537454